In [1]:
import tensorflow as tf
import pandas as pd
import random
import time
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.framework import arg_scope
import msvcrt
import sys
tf.reset_default_graph()

In [2]:
#데이터셋 경로 위치
DATASET_PATH      = "./diabetes.csv"
#텐서보드 저장경로
LOGS_PATH	= "logs/"
#모델의 입력 차원=> 현재 input pipeline에서 자동 조정 
NUM_DIM=8
NUM_LABELS		= 2
#batch size 모두 0 => input pipeline에서 자동 조정 
TRAIN_BATCH_SIZE	= 0
VALID_BATCH_SIZE	= 0
TEST_BATCH_SIZE		= 0
#epoch 
MAX_EPOCH=70

In [3]:
#(라벨,콘텐츠(etc 음성,텍스트)) 데이터 읽기	=> trainset,validset, test으로 나뉘어짐 / ratio는 다 합쳐서 1이 되어야 됌 
def read_label_file(file='./diabetes.csv',train_split_ratio=0.6,valid_split_ratio=0.2,test_split_ratio=0.2):
	df = pd.read_csv(file, encoding='latin-1')
	label=df.Outcome.tolist()
	contents=[]
	for row in df.iterrows():
		index, data = row
		contents.append(data[0:8].tolist())

	size=len(label)

	sum_ratio=train_split_ratio+valid_split_ratio+test_split_ratio
	
	assert  abs(1.0000 - sum_ratio) <= 0.0001, 'ratio의 합(%f)이 1이 아닙니다.'%(sum_ratio) 
    
	train_size=int(size*train_split_ratio)
	train_split_range=train_size
	train_contents=contents[0:train_split_range]
	train_label=label[0:train_split_range]
	
	valid_size=int(size*valid_split_ratio)
	valid_split_range=valid_size+train_split_range
	valid_contents=contents[train_split_range:valid_split_range]
	valid_label=label[train_split_range:valid_split_range]
	
	test_size=int(size*test_split_ratio)
	test_split_range=test_size+valid_split_range
	test_contents=contents[valid_split_range:test_split_range]
	test_label=label[valid_split_range:test_split_range]
	
	
	return train_size,valid_size,test_size,train_contents,train_label,valid_contents,valid_label,test_contents,test_label

def preprocess(data,label,num_class):

	#data를 sequences로 변환
	contents=data
	
	# string을 tensor로 변환, type=float32
	label = tf.one_hot(label,depth=num_class,on_value=1,off_value=0,axis=-1)
	return contents,label
		
def create_queue(contents,labels):

	#입력 큐 생성, 데이터 섞기(shuffle)=true 
	input_queue = tf.train.slice_input_producer(
												[contents, labels],
												shuffle=True)
	#입력 큐 대입(데이터,라벨)
	content = input_queue[0]
	label = input_queue[1]
	
	return content,label



In [4]:
#read_label_file_split(dataset_path)
total_train_size,total_valid_size,total_test_size,train_contents,train_labels,valid_contents,valid_labels,test_contents,test_labels = read_label_file(DATASET_PATH,0.6,0.2,0.2)
#batch size  = 전체 데이터셋 수의 10%으로 설정 
if(TRAIN_BATCH_SIZE==0):
    TRAIN_BATCH_SIZE=(int)(total_train_size*0.1)
if(VALID_BATCH_SIZE==0):
    VALID_BATCH_SIZE=(int)(total_valid_size*0.1) 
if(TEST_BATCH_SIZE==0):
    TEST_BATCH_SIZE=(int)(total_test_size*0.1) 

#train,valid,test set 각각 전처리
train_contents,train_labels=preprocess(train_contents,train_labels,NUM_LABELS)
valid_contents,valid_labels=preprocess(valid_contents,valid_labels,NUM_LABELS)
test_contents,test_labels=preprocess(test_contents,test_labels,NUM_LABELS)



#큐 생성  => train,valid,test  => train,valid,test set 각각 큐 생성 
train_content, train_label=create_queue(train_contents,train_labels)
valid_content, valid_label=create_queue(valid_contents,valid_labels)
test_content, test_label=create_queue(test_contents,test_labels)




# train,valid,test batch 객체 생성
train_batch = tf.train.batch(
                        [train_content, train_label],
                        batch_size=TRAIN_BATCH_SIZE
                    )

valid_batch = tf.train.batch(
                        [valid_content, valid_label],
                        batch_size=VALID_BATCH_SIZE
                    )		

test_batch = tf.train.batch(
                        [test_content,test_label],
                        batch_size=TEST_BATCH_SIZE
                    )			
print ("input pipeline ready")

input pipeline ready


In [5]:
#batch norm(배치 노멀라이제션)
def Batch_Norm(x, training, scope="bn"):
    with arg_scope([batch_norm],
                    scope=scope,
                    updates_collections=None,
                    decay=0.9,
                    center=True,
                    scale=True,
                    zero_debias_moving_mean=True):
        return tf.cond(training,
                        lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                        lambda : batch_norm(inputs=x, is_training=training, reuse=True))
                       
#BR layer(RB Layer 생성 = ReLU + Batch Norm)
def BR_Layer(x,name,output_num,training):
    xavier_initializer = tf.contrib.layers.xavier_initializer()
    shape = x.get_shape().as_list()
    input_num= shape[1]
    W = tf.Variable(xavier_initializer([input_num, output_num]))
    b = tf.Variable(xavier_initializer([output_num]))
    _x = tf.matmul(x, W) + b
    _x = Batch_Norm(_x,training, scope="bn_"+name)
    _x = tf.nn.relu(_x)
    return _x



# placeholder is used for feeding data.(데이터 입력)
x = tf.placeholder(tf.float32, shape=[None, NUM_DIM], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder(tf.float32, shape=[None, NUM_LABELS], name = 'y_target') # shape argument is optional, but this is useful to debug.
training=tf.placeholder(tf.bool)

xavier_initializer = tf.contrib.layers.xavier_initializer()

# reshape input data(입력 데이터 가공)
_x = tf.reshape(x,[-1,NUM_DIM],name="x_data")

#create a model(모델 생성)
_x=BR_Layer(_x,"layer1",32,training)
_x=BR_Layer(_x,"layer2",16,training)
_x=BR_Layer(_x,"layer3",32,training)

shape = _x.get_shape().as_list()
input_num= shape[1]                       
W = tf.Variable(xavier_initializer([input_num, NUM_LABELS]))
b = tf.Variable(xavier_initializer([NUM_LABELS]))
_x = tf.matmul(_x, W) + b
                       
pred=_x
prob_y=tf.nn.softmax(pred, name="prob_y")

# define the Loss function(비용함수 생성)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_target))

# define optimization algorithm(Adam optimizer 정의)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)


# correct_prediction is list of boolean which is the result of comparing(model prediction , data)(맞춘 수 계산 정의)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_target, 1))

#define accuracy(정확도 정의)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 

#save weight(웨이트 저장 정의)
t_vars = tf.trainable_variables()
saver = tf.train.Saver(max_to_keep=None,var_list=t_vars)
saver_def = saver.as_saver_def()

In [6]:
#gpu config(GPU 설정)
configure=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))

sess = tf.Session(config=configure) 

#define a tensorboard(텐서 보드 정의)
training_loss=tf.summary.scalar('training_loss', cost)
validation_loss=tf.summary.scalar('validation_loss', cost)

training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
validation_accuracy = tf.summary.scalar("validation_accuracy", accuracy)


# Merge all summaries into a single op
merged_op = tf.summary.merge_all()
writer=tf.summary.FileWriter(LOGS_PATH, sess.graph)

# initialization(초기화)
init_op = tf.global_variables_initializer()

In [7]:
print("Session started!")
start_session_time = time.time()
sess.run(init_op)

# 쓰레드 큐 초기화
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)
max_epoch=MAX_EPOCH
display_step=1
max_accuracy=0
max_index=0
#train(학습) 
for epoch in range(0,max_epoch+1):
    avg_cost = 0
    total_batch=(int)(total_train_size/TRAIN_BATCH_SIZE)
    for step in range(total_batch):
        _train_batch=sess.run([tf.cast(train_batch[0], tf.float32),tf.cast(train_batch[1], tf.float32)])
        if epoch!=0:
            _,c=sess.run([train_step,cost] , feed_dict={x: _train_batch[0], y_target: _train_batch[1], training: True})
            #loss 계산(학습 o)
            avg_cost += c / total_batch
        else:
            #loss 계산(학습 x)
            c=sess.run(cost , feed_dict={x: _train_batch[0], y_target: _train_batch[1], training: False})
            avg_cost += c / total_batch

    # 매 스텝마다 로그 출력
    if epoch % display_step == 0:

        print ("Epoch:", '%04d' % (epoch), "cost=","{:.9f}".format(avg_cost))

        _valid_batch=sess.run([tf.cast(valid_batch[0], tf.float32),tf.cast(valid_batch[1], tf.float32)])

        # traininig accuracy(학습셋 정확도)
        train_cos,train_acc, train_summ_acc,train_summ_loss = sess.run(
        [cost, accuracy, training_accuracy,training_loss], 
        feed_dict={x : _train_batch[0],  y_target : _train_batch[1],training: False})
        writer.add_summary(train_summ_acc, epoch) 
        writer.add_summary(train_summ_loss, epoch) 


        # validation accuracy(검증셋 정확도)
        valid_cos, valid_acc, valid_summ_acc,valid_summ_loss  = sess.run(
        [cost,accuracy, validation_accuracy,validation_loss],
        feed_dict={x: _valid_batch[0], y_target: _valid_batch[1],training: False})
        writer.add_summary(valid_summ_acc, epoch)
        writer.add_summary(valid_summ_loss, epoch)
        
        #save weight(웨이트 저장)
        saver.save(sess,"./weight/w",epoch)
        print("Train-Accuracy:", train_acc,"Train-Loss:", train_cos, "Validation-Accuracy:", valid_acc,"Val-Loss:", valid_cos,"\n")
        #model selection = vaildation accuracy(모델 선택 기준= 검증셋 정확도)
        if valid_acc > max_accuracy:
            max_accuracy=valid_acc
            max_index=epoch


#test accuracy(테스트 셋 정확도)
saver.restore(sess,"./weight/w-%d"%(max_index))
_test_batch=sess.run([tf.cast(test_batch[0],tf.float32),tf.cast(test_batch[1], tf.float32)])
_accuracy=sess.run(accuracy, feed_dict={x: _test_batch[0], y_target:  _test_batch[1],training: False})
print("(result)test accuracy: %g / weight-%d"%(_accuracy,max_index))    

# close thread qeueue, writer, session(모든 객체 종료)
coord.request_stop()
coord.join(threads)
writer.close()
sess.close()

print("close")

Session started!
Epoch: 0000 cost= 6.664971304
Train-Accuracy: 0.3478261 Train-Loss: 5.732937 Validation-Accuracy: 0.2 Val-Loss: 6.8177023 

Epoch: 0001 cost= 1.251628613
Train-Accuracy: 0.39130434 Train-Loss: 1.1713648 Validation-Accuracy: 0.06666667 Val-Loss: 1.6475751 

Epoch: 0002 cost= 1.216408736
Train-Accuracy: 0.36956522 Train-Loss: 1.132679 Validation-Accuracy: 0.06666667 Val-Loss: 1.6412461 

Epoch: 0003 cost= 1.194443023
Train-Accuracy: 0.41304347 Train-Loss: 1.1605194 Validation-Accuracy: 0.26666668 Val-Loss: 1.5236717 

Epoch: 0004 cost= 1.163214523
Train-Accuracy: 0.47826087 Train-Loss: 1.0068983 Validation-Accuracy: 0.26666668 Val-Loss: 1.3762877 

Epoch: 0005 cost= 1.148705971
Train-Accuracy: 0.39130434 Train-Loss: 1.1113482 Validation-Accuracy: 0.6 Val-Loss: 0.75704056 

Epoch: 0006 cost= 1.107149655
Train-Accuracy: 0.41304347 Train-Loss: 1.134908 Validation-Accuracy: 0.46666667 Val-Loss: 1.0282822 

Epoch: 0007 cost= 1.078127372
Train-Accuracy: 0.47826087 Train-Loss: 